In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import sys

# RNN
rnn.pdf / lec12.pdf / Lab12-RNN.pdf 참고

ex)

    I google at work

    I work at google

    google, work 단어의 생김새는 동일하지만 품사는 다름

    앞뒤 단어의 뜻에 따라 달라지는 품사

    RNN에서는 이것을 인식시킬수있다.(CNN에서는 못함)


RNN은 전체적인 맥락을 잡아내고 문장을 만들어낼수있다 

### Sequence data

자신한테서 출력된 값이 자신한테 입력되는 순환구조를 이룸

#### 출력 방향 2가지

    : output
        
        : 결과값 출력 
        
        : 다항분류에 사용   ex) 제목, 문장 만들기 

    : state
    
        : 옆에 있는 layer에게 영향을 주는 결과값 출력

        : 이전 값을 계속적으로 고려하여 계산을 진행해나갈수있다

        : 안에서 값을 변형해가면서 학습하기
        
        : 마지막에 나오는 state는 최종 결과값이 될 수 있다
        
        : 이진분류 중 감정분류 시에 많이 사용   ex) 행복한가, 부정적인가 긍정적인가
        
        
#### ex) hihello

입력데이터 : hihell

출력데이터 : ihello

같은 h 인데 그 뒤에 i가 올지 e가 올지 어떻게 예측하는가? 그 앞뒤를 보자


#### POS tagging

품사 구분 : output 을 보고서 명사, 형용사 구분을 학습시킴

가중치 계산에서는 옆 layer에서 넘어온 Wx 값도 같이 계산해줘야함 -> h = tanh(W1h0 + W2x2)

#### Sentiment Analysis

행복 분류를 state 최종 결과 보고 정하기

bptt(back propagation through time) : 시간에 따른 back propagation

## Vanila RNN
기본 알고리즘

In [3]:
from tensorflow.contrib import rnn

In [28]:
# hello 입력해서 출력하기 

tf.reset_default_graph()

# 입력값 준비 : hello
# one hot encoding : 숫자로 바꿔줘야 입력이 가능함

h = [1, 0, 0, 0] # 중복된거 빼면 4글자
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

# 셀 준비
cell = tf.contrib.rnn.BasicRNNCell(num_units=2) #hidden size 2개인 이유는...??? 01이라서?# 두개로 잡았으니 결과 2개로 나옴

# 입력데이터 준비
x_data = np.array([[h, e, l, l, o]], dtype=np.float32) 
    # sequence의 길이가 됨  # 입력데이터 총 3차원(문자열 최대 차원)
print(x_data)
print("----------------------------------------------")

# rnn 실행
outputs, _state = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32) 
                                    # 실행시킬거, 입력데이터, type  # 2개의 결과 변수로 선언
    
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print(sess.run(outputs)) # 각각의 시퀀스에 따른 결과 
                         # h, e, l. l. o 순으로 나온 결과값

print("----------------------------------------------")

print(sess.run(_state))
# 마지막으로 나온 output 과 state의 결과값은 같아야함 

[[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]]
----------------------------------------------
[[[ 0.1084308   0.23005372]
  [-0.44041133 -0.59082305]
  [-0.46746534  0.61441123]
  [-0.5914525   0.64826185]
  [-0.28519422 -0.5536942 ]]]
----------------------------------------------
[[-0.28519422 -0.5536942 ]]


In [27]:
# 품사 구분하도록 입력하기

tf.reset_default_graph()
tf.set_random_seed(777)

# 입력값 준비
# one hot encoding 시키기
# 문장1 : I work at google = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
# 문장2 : I google at work = [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]]


# 입력데이터 준비 : 문자 데이터 3차원으로 만들기
inputs = np.array([[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]],
                  [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]]]) # 각각의 데이터를 3차원으로 만들어주기

tf_inputs = tf.constant(inputs, dtype=tf.float32)

# 셀준비
cell = tf.contrib.rnn.BasicRNNCell(num_units=3) # 몇개로 출력하는지에 대해 지정하는지에 따라서 음.. 중요하진 않아요^^

# rnn 실행
outputs, _state = tf.nn.dynamic_rnn(cell, inputs=tf_inputs, dtype=tf.float32) 

# session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print(sess.run(outputs))
# 문장1과 문장2 비교
# I  : 처음 시작값 똑같이 나옴 이니까
# at : 앞(과거)에서부터 받은 가중치 값이 다르므로 다른 값을 가짐 

print("------------------------------------------------")

print(sess.run(_state))
# 각각의 state값은 outputs의 마지막 출력값과 동일

print("------------------------------------------------")

# 가중치 값에 따라 shape이 어떻게 잡히는지 알아보기....??
variable_names = [v.name for v in tf.trainable_variables()]
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)
                # 가중치 값까지 알아볼 필요는 없어용..........

[[[-0.50944704  0.33166462  0.6126557 ]
  [-0.20793891  0.24406303 -0.75278705]
  [-0.06346128 -0.52844936  0.68356085]
  [-0.36491966  0.8857268  -0.02324395]]

 [[-0.50944704  0.33166462  0.6126557 ]
  [-0.30707452  0.62735885  0.21719742]
  [ 0.5043804  -0.14038289  0.3744523 ]
  [-0.11641277  0.70696247 -0.7512605 ]]]
------------------------------------------------
[[-0.36491966  0.8857268  -0.02324395]
 [-0.11641277  0.70696247 -0.7512605 ]]
------------------------------------------------
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


In [45]:
# hihello 학습 과정
# h가 입력되면 예측되는 값은 i / i가 입력되면 예측되는 값은 h ..로 나와야한다

tf.reset_default_graph()
tf.set_random_seed(777)

# 중복된 단어빼고 숫자화 시키기
idx2char = ['h', 'i', 'e', 'l', 'o']

# one-hot encoding
x_one_hot = [[[1, 0, 0, 0, 0],       # h 0  
              [0, 1, 0, 0, 0],       # i 1 
              [1, 0, 0, 0, 0],       # h 0
              [0, 0, 1, 0, 0],       # e 2
              [0, 0, 0, 1, 0],       # l 3
              [0, 0, 0, 1, 0],       # l 3
              [0, 0, 0, 0, 1]]]      # 0 4

# 입력 데이터
x_data = [[0, 1, 0, 2, 3, 3]]
          # h, i, h, e, l, l

# 출력 데이터
y_data = [[0, 1, 0, 2, 3, 3, 4]]
         # h, i, h, e, l, l, o 
    
# X, y 변수 준비
    # [batch_size(입력값의 차원의 개수), sequence length, input dimension]
    # 각각의 값 변수에 담기 
    # [batch_size(입력값의 차원의 개수), sequence length, input dimension] 
    # hi끼리 묶고, ih끼리 묶고 등등.. 이렇게 나눠서 입력하지 않으므로 batch size는 1임
    # hihell 6개 넣고 ihello 6개 나오니 sequence length 6
    # 중복된 글자 빼고 h, i, e, l, o -> input dimension 5
    
num_classes = 5
input_dim = 5 
hidden_size = 5 # num_units 와 같은 뜻
batch_size = 1
sequence_len = 7
learning_rate = 0.1

X = tf.placeholder(tf.float32, [None, sequence_len, input_dim]) # batch_size 부분 무슨값 None으로 처리 
y = tf.placeholder(tf.int32, [None, sequence_len]) # 결과 입력한 개수만큼 나옴

# rnn 모델 작성
## 셀준비
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size) 
## 초기값 지정
initial_state = cell.zero_state(batch_size, tf.float32)  # cell안에있는 각각의 값을 0으로 청소해주기
## rnn 실행
outputs, _state = tf.nn.dynamic_rnn(cell=cell, inputs=X, dtype=tf.float32,
                                   initial_state=initial_state) # 초기값 넣어줘서 더 깨끗한 값 만들어내자요오오오옹

# 평면화
X_for_fc = tf.reshape(outputs, [-1, hidden_size]) 
            # 2차원으로 만들어주기위해 reshape 사용 / flatten은 걍 1차원으로 만들어버리니까
    
# FC
## W, b, logit 준비하기 / 한줄에 작성해주는 함수 
outputs = tf.contrib.layers.fully_connected(inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)
                                            # 입력값, 출력개수, activation_function = None 안쓸꺼야....
    
outputs = tf.reshape(outputs, [batch_size, sequence_len, num_classes])
# activation function을 통과시키지 않은 output이 logit인데 비용계산 부분에 넣을떄는 3차원으로 넣어야한다 
# 그렇기에 크기 재조정해줘야한다
weights = tf.ones([batch_size, sequence_len])
# 가중치 다 1로 채워서 넘겨준다 왜?.......ㅎ

# 비용 계산
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = y, weights = weights) 
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# 예측값
prediction = tf.argmax(outputs, 2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 훈련횟수
    for i in range(50):
        _, cost = sess.run([train, loss], feed_dict={X:x_one_hot, y:y_data})
        result = sess.run(prediction, feed_dict={X:x_one_hot})
        print(i, "loss:", cost, "       prediction:", result, "       true Y : ", y_data)
        # 비용이 잘 줄어드는가 # 예측값과 실제값이 잘 맞는가 / 한 3번째부터 맞기시작한것을 볼수있음
        
    # 결과값 문자로 출력해보자
    result_str = [idx2char[c] for c in np.squeeze(result)] # np.squeeze : 현재 차원에서 한 차원줄이는 함수
    print("\n 결과 :", ", ".join(result_str))

0 loss: 1.638345        prediction: [[0 3 3 3 3 3 2]]        true Y :  [[0, 1, 0, 2, 3, 3, 4]]
1 loss: 1.386576        prediction: [[0 0 0 3 3 3 4]]        true Y :  [[0, 1, 0, 2, 3, 3, 4]]
2 loss: 1.1635329        prediction: [[0 0 0 3 3 3 4]]        true Y :  [[0, 1, 0, 2, 3, 3, 4]]
3 loss: 0.93181574        prediction: [[0 1 0 2 3 3 4]]        true Y :  [[0, 1, 0, 2, 3, 3, 4]]
4 loss: 0.70485556        prediction: [[0 1 0 2 3 3 4]]        true Y :  [[0, 1, 0, 2, 3, 3, 4]]
5 loss: 0.50350326        prediction: [[0 1 0 2 3 3 4]]        true Y :  [[0, 1, 0, 2, 3, 3, 4]]
6 loss: 0.3518529        prediction: [[0 1 0 2 3 3 4]]        true Y :  [[0, 1, 0, 2, 3, 3, 4]]
7 loss: 0.2512133        prediction: [[0 1 0 2 3 3 4]]        true Y :  [[0, 1, 0, 2, 3, 3, 4]]
8 loss: 0.18412565        prediction: [[0 1 0 2 3 3 4]]        true Y :  [[0, 1, 0, 2, 3, 3, 4]]
9 loss: 0.13794506        prediction: [[0 1 0 2 3 3 4]]        true Y :  [[0, 1, 0, 2, 3, 3, 4]]
10 loss: 0.10591277        prediction

#### gradient vanishing : 값이 너무 작아져 측정할 수가 없음

#### gradient exploding : 값이 커져버리면 수렴이 안됨 최저비용에 도착을 못함


이것들을 고안해서 나온 방법▼

### LSTM(long sequence training memory)

새로운 state가 하나 더 출력이 된다 전첵적인 출력의 개수는 3 <- Vanila와의 차이점

forget gate : 새로운 기억이 들어갈 곳을 남겨두기 위해 잔여 데이터 기억을 몇% 지운다

하나의 memory cell에는 과거의 기억과 새로운 기억이 같이 존재할 수 있도록 한다

memory cell : 기억해야할 것들을 따로 보관하는 형태

두가지 상태선을 가지고 처리한당용ㄻㅇ란이런아ㅏ

- http://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [65]:
tf.reset_default_graph()
tf.set_random_seed(777)

inputs = np.array([[[1, 0]]])

tf_inputs = tf.constant(inputs, dtype=tf.float32)
cell = tf.nn.rnn_cell.LSTMCell(num_units=1)
outputs, _states = tf.nn.dynamic_rnn(cell=cell, dtype=tf.float32, inputs=tf_inputs)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_run, states_run = sess.run([outputs, _states])
    print("output values")
    print(outputs_run)
    print("\nmemory cell value ")
    print(states_run.c)
    print("\nhidden state value ")
    print(states_run.h)

output values
[[[0.09927537]]]

memory cell value 
[[0.18134572]]

hidden state value 
[[0.09927537]]


In [66]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the thiadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]

df = pd.DataFrame(paragraph_dict_list)
df = df[["paragraph", "category"]]
df

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food
5,the garlic fries were a great starter (and a h...,food
6,our meal was excellent i had the pasta ai form...,food
7,what i enjoy most about palo alto is so many r...,food
8,the drinks came out fairly quickly a good two ...,food
9,despite the not so good burger the service was...,food


In [67]:
results = set()
df["paragraph"].str.lower().str.split().apply(results.update)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: paragraph, dtype: object

In [68]:
results

{'(and',
 '1',
 '1.5',
 '1.5x',
 '10-9',
 '14',
 '2',
 '2014',
 '33/1',
 'a',
 'about',
 'action',
 'after',
 'against',
 'ago',
 'ai',
 'alisson',
 'all',
 'almost',
 'also',
 'although',
 'alto',
 'amazing',
 'american',
 'and',
 'angelique',
 'any',
 'apps',
 'are',
 'area',
 'around',
 'as',
 'at',
 'australia',
 'avoided',
 'azarenka',
 'back',
 'battle',
 'be',
 'beef',
 'been',
 'before',
 'began',
 'began—too',
 'being',
 'better',
 'between',
 'big',
 'biscuits',
 'bit',
 'bookmark',
 'bookmarked',
 'both',
 'bottom',
 'brazilian',
 'bread',
 'breakthroughs',
 'briton',
 'brunch',
 'burger',
 'burgers',
 'business',
 'busy',
 'but',
 'by',
 'came',
 'campaigns',
 'can',
 'card',
 'caroline',
 'case',
 'certainly',
 'champion',
 'champions',
 'chance',
 'chances',
 'chicken',
 'city',
 'clay',
 'clean',
 'commented',
 'competition',
 'complain',
 'confidence',
 'contrast',
 'could',
 'course',
 'courteous',
 'courts',
 'coworkers',
 'crisp',
 'croatia',
 'crumbs',
 'culminating

In [69]:
# 단어 인덱스로 바꾸기
idx2word = dict(enumerate(results))
idx2word

# 잘만들어졌는지 확인해보자
# 인덱스로 문자 알려주기
idx2word[396]
idx2word[100]

'burgers'

In [70]:
# tuple 형태로 바꾸기 
word2idx = {v:k for k, v in idx2word.items()}
word2idx

# 잘 만들었는지 확인해보기
# 문자로 인덱스 알려주기 
word2idx["bread"]

38

In [71]:
def encode_paragraph(paragraph):
    words = paragraph.split(" ")
    encoded = []
    for word in words:
        encoded.append([word2idx[word]])
    
    return encoded

def encode_category(category):
    if category == "food":
        return [1, 0]
    else:
        return [0, 1]

# 전체 단어의 개수 몇개인지 세어주는 함수 
def word_cnt(paragraph):
    return len(paragraph.split(" "))

In [72]:
# 컬럼 새로 만들기 # 훈련시키려면 문자들 다 숫자로 바꿔야하니까 바꾼거 
df["enc_paragraph"] = df.paragraph.apply(encode_paragraph)
df["enc_category"] = df.category.apply(encode_category)
df["seq_length"] = df.paragraph.apply(word_cnt)

In [73]:
# 확인
df

,paragraph,category,enc_paragraph,enc_category,seq_length
0,dishplace is located in sunnyvale downtown the...,food,"[[130], [336], [318], [60], [505], [346], [120...","[1, 0]",53
1,service can be slower during busy hours but ou...,food,"[[36], [482], [478], [150], [174], [210], [161...","[1, 0]",19
2,portions are huge both french toast and their ...,food,"[[510], [344], [342], [2], [329], [184], [57],...","[1, 0]",42
3,we started with apps going the chicken and waf...,food,"[[391], [499], [469], [463], [118], [124], [26...","[1, 0]",43
4,the biscuits and gravy was too salty two peopl...,food,"[[124], [359], [57], [512], [328], [506], [111...","[1, 0]",82
5,the garlic fries were a great starter (and a h...,food,"[[124], [121], [415], [213], [396], [330], [50...","[1, 0]",24
6,our meal was excellent i had the pasta ai form...,food,"[[422], [275], [328], [21], [489], [236], [124...","[1, 0]",50
7,what i enjoy most about palo alto is so many r...,food,"[[106], [489], [494], [312], [261], [54], [360...","[1, 0]",43
8,the drinks came out fairly quickly a good two ...,food,"[[124], [284], [474], [6], [283], [41], [396],...","[1, 0]",49
9,despite the not so good burger the service was...,food,"[[507], [124], [122], [40], [381], [266], [124...","[1, 0]",82


지금까지의 sample 데이터는 길이가 다 같았음

길이가 서로 다를 경우에는 우찌?

길이가 모자란 애들은 공백으로 처리되어 공백도 하나의 데이터로 분석을 해버리고 성능은 떨어지게 된다

그래서 쓰는게 

# 

#### dynamic_rnn : 불규칙한 입력데이터의 길이를 동적으로 처리하는 함수

static_rnn은 다 같은 길이로 쓰는 애 

그렇기 때문에 dynamic_rnn을 많이 씀

#

가장 긴 길이를 기준으로 하여 크기를 잡아줌

공백이 생기는 애들은 공백에 padding 해줌 즉, 표시 해두기 

그럼 dynamic_rnn은 학습을 안해도 되는구나 하고 빼버림

그래서 데이터의 전체 길이를 알아야했다요!!!!!!!!!!

#

그니까 최대값이 무엇인지 알아내는 코드가 필요하다....

In [85]:
# 문장의 최대 길이 알아내기 

max_word_cnt = 0
for row in df["paragraph"]:
    if len(row.split(" ")) > max_word_cnt:
        max_word_cnt = len(row.split(" "))
        
print(max_word_cnt)

91


In [86]:
# 문장의 길이가 91개가 안될 경우 -1로 패딩 처리
def sequence_padding(enc_paragraph):
    seq_length = len(enc_paragraph)
    for i in range(seq_length, max_word_cnt):
        enc_paragraph.append([-1])
        
    return enc_paragraph

In [87]:
df["enc_paragraph"] = df.enc_paragraph.apply(sequence_padding)

In [93]:
df

,paragraph,category,enc_paragraph,enc_category,seq_length
0,dishplace is located in sunnyvale downtown the...,food,"[[130], [336], [318], [60], [505], [346], [120...","[1, 0]",53
1,service can be slower during busy hours but ou...,food,"[[36], [482], [478], [150], [174], [210], [161...","[1, 0]",19
2,portions are huge both french toast and their ...,food,"[[510], [344], [342], [2], [329], [184], [57],...","[1, 0]",42
3,we started with apps going the chicken and waf...,food,"[[391], [499], [469], [463], [118], [124], [26...","[1, 0]",43
4,the biscuits and gravy was too salty two peopl...,food,"[[124], [359], [57], [512], [328], [506], [111...","[1, 0]",82
5,the garlic fries were a great starter (and a h...,food,"[[124], [121], [415], [213], [396], [330], [50...","[1, 0]",24
6,our meal was excellent i had the pasta ai form...,food,"[[422], [275], [328], [21], [489], [236], [124...","[1, 0]",50
7,what i enjoy most about palo alto is so many r...,food,"[[106], [489], [494], [312], [261], [54], [360...","[1, 0]",43
8,the drinks came out fairly quickly a good two ...,food,"[[124], [284], [474], [6], [283], [41], [396],...","[1, 0]",49
9,despite the not so good burger the service was...,food,"[[507], [124], [122], [40], [381], [266], [124...","[1, 0]",82


In [88]:
print(df["enc_paragraph"][0])

[[130], [336], [318], [60], [505], [346], [120], [336], [375], [171], [124], [164], [179], [101], [482], [478], [525], [456], [447], [174], [468], [73], [161], [430], [37], [57], [489], [474], [456], [27], [303], [224], [134], [389], [396], [43], [519], [213], [347], [210], [40], [489], [185], [35], [279], [66], [377], [127], [181], [124], [479], [456], [442], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]


In [94]:
# 입력값들을 배열로 변환
enc_paragraph = np.array(df.enc_paragraph.tolist())
enc_category = np.array(df.enc_category.tolist())
seq_length = np.array(df.seq_length.tolist())

In [95]:
train_X = enc_paragraph
train_y = enc_category

In [96]:
print(train_X.shape)
print(train_y.shape)

(20, 91, 1)
(20, 2)


In [97]:
# 모델 구축
tf.reset_default_graph()
tf.set_random_seed(777)

learning_rate = 0.001
n_epochs = 300

X = tf.placeholder(tf.float32, [None, max_word_cnt, 1])
y = tf.placeholder(tf.int32, [None, 2])

embedding = tf.layers.dense(X, 5)

cell = tf.nn.rnn_cell.LSTMCell(num_units=64)
output, state = tf.nn.dynamic_rnn(cell, embedding, dtype=tf.float32, sequence_length=seq_length)

dense_layer = tf.layers.dense(state.h, 32)
logits = tf.layers.dense(dense_layer, 2)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [98]:
print(X)
print(embedding)
print(state)
print(dense_layer)
print(logits)

Tensor("Placeholder:0", shape=(?, 91, 1), dtype=float32)
Tensor("dense/BiasAdd:0", shape=(?, 91, 5), dtype=float32)
LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 64) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 64) dtype=float32>)
Tensor("dense_1/BiasAdd:0", shape=(?, 32), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 2), dtype=float32)


In [99]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        _, cost = sess.run([train, loss], feed_dict={X:train_X, y:train_y})
        if epoch % 50 == 0:
            pred = tf.nn.softmax(logits)
            correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_pred, "float"))
            cur_acc = accuracy.eval({X:train_X, y:train_y})
            print("epoch:" + str(epoch) + ", loss:" + str(cost) + ", accuracy:" + str(cur_acc))

epoch:0, loss:0.8909272, accuracy:0.5
epoch:50, loss:0.45044193, accuracy:0.75
epoch:100, loss:0.15099776, accuracy:0.95
epoch:150, loss:0.02503803, accuracy:1.0
epoch:200, loss:0.0032332118, accuracy:1.0
epoch:250, loss:0.00103654, accuracy:1.0
